In [51]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


data_dir_1 = '/workspaces/Final-Year-Project/Data/2023/'

file_list = os.listdir(data_dir_1)

sas_files = [file for file in file_list if file.endswith('.XPT') or file.endswith('.xpt')]

data_frames = {}

for file_name in sas_files:
    file_path = os.path.join(data_dir_1, file_name)
    
    df = pd.read_sas(file_path, format='xport', encoding='iso-8859-1')
    
    data_frames[file_name] = df

merged_df = data_frames[sas_files[0]]  # Start with the first dataset

for file_name in sas_files[1:]:  # Skip the first file as it's already merged
    merged_df = pd.merge(merged_df, data_frames[file_name], on='SEQN', how='outer')  # Merge based on SEQN

merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11933 entries, 0 to 11932
Columns: 290 entries, SEQN to RHQ332
dtypes: float64(283), object(7)
memory usage: 26.4+ MB


In [52]:


# Now select only float64 columns
df_float = merged_df.select_dtypes(include=['float64'])

# Display the cleaned dataframe info
df_float.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11933 entries, 0 to 11932
Columns: 283 entries, SEQN to RHQ332
dtypes: float64(283)
memory usage: 25.8 MB


In [53]:
merged_df_cleaned = df_float.dropna(subset=['LBXGH', 'LBXGLU'], how='all')

def categorize_diabetes(row):
    if (pd.notnull(row['LBXGH']) and row['LBXGH'] >= 6.5) or (pd.notnull(row['LBXGLU']) and row['LBXGLU'] >= 126):
        return 2
    
    elif (pd.notnull(row['LBXGH']) and 5.7 <= row['LBXGH'] < 6.5) or (pd.notnull(row['LBXGLU']) and 100 <= row['LBXGLU'] < 126):
        return 1
    else:
        return 0

merged_df_cleaned['Diabetes Status'] = merged_df_cleaned.apply(categorize_diabetes, axis=1)

merged_df_cleaned = merged_df_cleaned.drop(columns=['LBXGH', 'LBXGLU'])

df = merged_df_cleaned
df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 6717 entries, 0 to 11932
Columns: 282 entries, SEQN to Diabetes Status
dtypes: float64(281), int64(1)
memory usage: 14.5 MB


/tmp/ipykernel_16796/2265844166.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_cleaned['Diabetes Status'] = merged_df_cleaned.apply(categorize_diabetes, axis=1)


In [54]:
df = df.dropna(subset=['Diabetes Status'])

missing_percentage = df.isnull().mean() * 100

columns_to_drop = missing_percentage[missing_percentage > 10].index

df = df.drop(columns=columns_to_drop)

df = df.dropna()

df.info()





<class 'pandas.core.frame.DataFrame'>
Index: 4701 entries, 0 to 11932
Data columns (total 71 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SEQN             4701 non-null   float64
 1   IMQ011           4701 non-null   float64
 2   OHQ845           4701 non-null   float64
 3   OHQ620           4701 non-null   float64
 4   OCD150           4701 non-null   float64
 5   WHD010           4701 non-null   float64
 6   WHD020           4701 non-null   float64
 7   WHD050           4701 non-null   float64
 8   WHQ070           4701 non-null   float64
 9   HOD051           4701 non-null   float64
 10  MCQ010           4701 non-null   float64
 11  AGQ030           4701 non-null   float64
 12  MCQ053           4701 non-null   float64
 13  HUQ010           4701 non-null   float64
 14  HUQ030           4701 non-null   float64
 15  HUQ055           4701 non-null   float64
 16  HUQ090           4701 non-null   float64
 17  SLD012           4

In [55]:
df['Diabetes Status'].value_counts()

Diabetes Status
0    2284
1    1800
2     617
Name: count, dtype: int64

In [56]:
from sklearn.utils import resample

no_diabetes = df[df['Diabetes Status'] == 0]
prediabetes = df[df['Diabetes Status'] == 1]
diabetes = df[df['Diabetes Status'] == 2]

minority_class_size = len(diabetes)

no_diabetes_downsampled = resample(no_diabetes, replace=False, n_samples=minority_class_size, random_state=42)
prediabetes_downsampled = resample(prediabetes, replace=False, n_samples=minority_class_size, random_state=42)

df_balanced = pd.concat([no_diabetes_downsampled, prediabetes_downsampled, diabetes])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

df_balanced['Diabetes Status'].value_counts()

Diabetes Status
0    617
2    617
1    617
Name: count, dtype: int64

In [57]:
# Calculate the correlation matrix
df_corr = df_balanced.corr()

# Filter out the correlations with Diabetes Status, and get those greater than 0.20
df_top_filtered = df_corr['Diabetes Status'].abs()[df_corr['Diabetes Status'].abs() > 0.20]

# Now, get the list of column names that have a correlation greater than 0.20
columns_to_keep = df_top_filtered.index

# Select only the columns with correlation > 0.20
df_filtered = df_balanced[columns_to_keep]

# Save the filtered dataframe to CSV
df_filtered.to_csv('/workspaces/Final-Year-Project/Cleaned Data/TopCorrelationData.csv', index=False)

# Verify the new dataframe info
df_top_filtered.sort_values(ascending=False)



Diabetes Status    1.000000
BMXWAIST           0.420927
RIDAGEYR           0.410169
RXQ033             0.350545
BPQ020             0.345960
HUQ010             0.340176
BMXBMI             0.321933
BMXWT              0.284921
BPQ101D            0.281895
BMXARMC            0.270018
OHQ845             0.232009
BPAOCSZ            0.229840
BMXHIP             0.226191
OCD150             0.214947
BPXOSY2            0.206634
Name: Diabetes Status, dtype: float64

In [58]:
from sklearn.model_selection import train_test_split

df_train_test, df_validate = train_test_split(df_filtered, test_size=0.1, random_state=42)

df_validate.to_csv('/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv', index=False)

df_train_test.to_csv('/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv', index=False)

In [59]:
df_train_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1665 entries, 344 to 1126
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   OHQ845           1665 non-null   float64
 1   OCD150           1665 non-null   float64
 2   HUQ010           1665 non-null   float64
 3   RIDAGEYR         1665 non-null   float64
 4   BPAOCSZ          1665 non-null   float64
 5   BPXOSY2          1665 non-null   float64
 6   BMXWT            1665 non-null   float64
 7   BMXBMI           1665 non-null   float64
 8   BMXARMC          1665 non-null   float64
 9   BMXWAIST         1665 non-null   float64
 10  BMXHIP           1665 non-null   float64
 11  BPQ020           1665 non-null   float64
 12  BPQ101D          1665 non-null   float64
 13  RXQ033           1665 non-null   float64
 14  Diabetes Status  1665 non-null   int64  
dtypes: float64(14), int64(1)
memory usage: 208.1 KB
